# Helloworld Extraction of BYOC imagery via Process API Client

In [1]:
import os

# define repo name and get root working directory
repo = 'byoc-api'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
root_path

'C:\\Users\\crwil\\Documents\\GitHub\\sentinelhub\\byoc-api'

### _Setup imports and configuration_

In [2]:
from sentinelhub import (
    SHConfig, DataCollection, Geometry, BBox, CRS,
    SentinelHubRequest, filter_times, bbox_to_dimensions, MimeType,
    SentinelHubBYOC, ByocCollection, ByocTile, ByocCollectionAdditionalData,
    DownloadFailedException
)

# Initialize SentinelHubBYOC class
config = SHConfig()
byoc = SentinelHubBYOC(config=config)

In [3]:
collections_iterator = byoc.iter_collections()
my_collections = list(collections_iterator)
my_collections

[{'id': '2c709ccd-12bd-42c1-8308-ff8345e4a0d0',
  'userId': '1f046949-4e02-4859-a4c1-807fc312dc7a',
  'name': 'another_new_collection',
  's3Bucket': '4rd-climate-finance',
  'additionalData': {'bands': {'LC09_L2SP_203023_20220310_20220312_02_T1_SR_B1': {'bitDepth': 16,
     'source': 'LC09_L2SP_203023_20220310_20220312_02_T1_SR_B1',
     'bandIndex': 1,
     'sampleFormat': 'UINT'},
    'LC09_L2SP_203023_20220310_20220312_02_T1_SR_B2': {'bitDepth': 16,
     'source': 'LC09_L2SP_203023_20220310_20220312_02_T1_SR_B2',
     'bandIndex': 1,
     'sampleFormat': 'UINT'}},
   'maxMetersPerPixel': 2400.0,
   'extent': {'type': 'Polygon',
    'coordinates': [[[-3.789221674, 51.994693002],
      [-3.789221674, 54.177840954],
      [-0.164235984, 54.177840954],
      [-0.164235984, 51.994693002],
      [-3.789221674, 51.994693002]]]},
   'hasSensingTimes': 'NO'},
  'noData': 0,
  'created': '2022-03-17T20:26:55.456140Z',
  'requiresMetadataUpdate': False},
 {'id': '3d2a9be6-6c5b-4e7a-87d7-e6ea3

In [4]:
# get pathname to configuration file
cfg_file = os.path.join( os.path.join( root_path, 'cfg' ), 'simple-timeseries.yml' )
with open( cfg_file ) as f:
    print ( f.read() )

request:
    evalscript: |
                //VERSION=3
                function setup() {
                    return {
                      input: [ "B1", "B2" ],
                      output: {
                          bands: 2,
                          sampleType: "UINT16"
                      }
                    };                  
                }
                function evaluatePixel(sample) {
                    return [ sample.B1, sample.B2 ];
                }
    inputs:
        - collection: byoc-3d2a9be6-6c5b-4e7a-87d7-e6ea30e72f78
responses:
    default: TIFF



In [5]:
import yaml
from munch import munchify

# load cfg file using yaml parser
with open( cfg_file, 'r' ) as f:
    config = munchify( yaml.safe_load( f ) )

### _Define Area of Interest collocated with Isle of Portland_

In [6]:
# setup aoi limits and resolution
coords = 530575, 5595700, 547118, 5606883
resolution = 30

In [7]:
# create instance of shclient class
from processapi import Client
client = Client( config )

In [8]:
# get utm bbox
bbox = client.getBoundingBox( coords, src_crs=CRS(32630) )
print ( bbox, bbox.crs.value )

530575.0,5595700.0,547118.0,5606883.0 32630


In [9]:
# get aoi image dimensions
client.getBoxDimensions( bbox, resolution )

(551, 373)

In [10]:
# pick a few dates
from datetime import datetime
timeframe = { 'start' : datetime( 2021, 9, 1, 0, 0, 0 ), 
                'end' : datetime( 2021, 11, 7, 23, 59, 59 ) }
timeframe

{'start': datetime.datetime(2021, 9, 1, 0, 0),
 'end': datetime.datetime(2021, 11, 7, 23, 59, 59)}

### _Validate BYOC collection timestamps and imagery accessible via Process API Client_

In [11]:
# get S1 acquisitions between start / end dates satisfying filter conditions
client.getDatasetTimeStamps( config.request.inputs[ 0 ], bbox, timeframe )

[datetime.datetime(2021, 9, 7, 0, 0, tzinfo=tzutc())]

In [12]:
# get time series
response = client.getTimeSeries ( bbox, timeframe, resolution )
response._df

,default,time
0,"[[[7320, 7540], [7303, 7530], [7308, 7551], [7...",2021-09-07 00:00:00+00:00
